<a href="https://colab.research.google.com/github/pgurazada/fast-fast-ai/blob/master/unsupervised_amazon_review_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install contractions

In [0]:
!git clone https://github.com/facebookresearch/fastText.git

In [0]:
%cd fastText

In [0]:
!make

In [0]:
!pip install .

In [0]:
import pandas as pd
import numpy as np

In [0]:
import re, string, unicodedata
import nltk
import contractions
import inflect

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from fastText import load_model

from sklearn.cluster import SpectralClustering
from sklearn.feature_extraction import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from tqdm import tqdm

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive', force_remount=True)

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [0]:
data_df = pd.read_csv('/content/gdrive/My Drive/data-for-experimentation/amazon-reviews-corpus.csv')

In [0]:
samples = data_df['text'].apply(str).values

In [0]:
p = inflect.engine()
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [0]:
cleaned_samples = []

In [0]:
for sample in tqdm(samples):
  cleaned_sample = []
  
  words = nltk.word_tokenize(sample)
  
  for word in words:
    word = re.sub('\[[^]]*\]', '', word.lower())
    word = re.sub(r'[^\w\s]', '', word)
    word = contractions.fix(word)
    
    if word.isdigit():
      word = p.number_to_words(word)
    
    word = stemmer.stem(word)
    word = lemmatizer.lemmatize(word, pos='v')
      
    if word and word not in stopwords.words('english'):
      cleaned_sample.append(word)
      
  cleaned_samples.append(' '.join(cleaned_sample))

In [0]:
with open('/content/gdrive/My Drive/data-for-experimentation/amazon-reviews-cleaned.txt', 'w') as f:
    for sample in cleaned_samples:
        f.write("%s\n" % sample)

In [0]:
! ./fasttext skipgram \
-input /content/gdrive/My\ Drive/data-for-experimentation/amazon-reviews-cleaned.txt \
-output /content/gdrive/My\ Drive/data-for-experimentation/amazon-skipgram-model \
-epoch 200

In [0]:
embedding_model = load_model('/content/gdrive/My Drive/data-for-experimentation/amazon-skipgram-model.bin')

In [0]:
words, embeddings = [], []

In [0]:
for word in tqdm(embedding_model.get_words()):
  words.append(word)
  embeddings.append(embedding_model.get_word_vector(word))

In [0]:
embeddings = np.array(embeddings, dtype='float')

In [0]:
cluster_model = SpectralClustering(assign_labels='discretize', random_state=20130810, n_jobs=-1)

In [0]:
cluster_model.fit(embeddings)

Does clustering tf-idf important words improve clusters?

In [0]:
cv = CountVectorizer(max_df=0.85, stop_words=stopwords.words('english'))
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)

In [0]:
word_count_vector = cv.fit_transform(cleaned_samples)
tfidf_transformer.fit(word_count_vector)